# **Multi-Layer Perceptron (MLP) for Regression**

This notebook provides a comprehensive implementation of a Multi-Layer Perceptron (MLP) neural network for regression tasks. It covers weight initialization, forward propagation, backpropagation for gradient computation, and the training process using mini-batch Gradient Descent. The Mean Squared Error (MSE) is used as the loss function, and the sigmoid function is employed as the activation for the hidden layer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Included for consistency, though not directly used in 1D MLP visualization

np.random.seed(42) # Set random seed for reproducibility


# **1. Helper Functions**

This section defines essential helper functions required for the MLP: the sigmoid activation function and its derivative, and the Mean Squared Error (MSE) cost function. Optional L1 and L2 regularization terms are also included, though they are commented out in the main training loop by default.

In [ ]:
def MLP_sigmoid(z):
    """
    Computes the sigmoid activation function.
    
    Args:
        z (np.array): Input to the sigmoid function (scalar, vector, or matrix).
        
    Returns:
        np.array: Output of the sigmoid function, element-wise.
    """
    return 1.0 / (1.0 + np.exp(-z))

def MLP_sigmoid_derivative(rZ):
    """
    Computes the derivative of the sigmoid function.
    
    Args:
        rZ (np.array): Input to the sigmoid function (pre-activation values).
        
    Returns:
        np.array: Derivative of the sigmoid, element-wise, at rZ.
    """
    sigma_of_rZ = MLP_sigmoid(rZ)
    return sigma_of_rZ * (1.0 - sigma_of_rZ)

def MLP_MSE_cost(y_true, y_pred, model, W1, W2):
    """
    Computes the Mean Squared Error (MSE) cost function.
    
    Args:
        y_true (np.array): True target values (1xBatchSize or BatchSize x 1).
        y_pred (np.array): Predicted output values (1xBatchSize).
        model (dict): Dictionary containing model parameters (e.g., regularization lambdas).
        W1 (np.array): Weights matrix for the hidden layer.
        W2 (np.array): Weights matrix for the output layer.
        
    Returns:
        float: The scalar MSE cost.
    """
    # Ensure y_true has the same shape as y_pred for element-wise operations
    if y_true.ndim == 1:
        y_true = y_true.reshape(1, -1)

    cost = np.sum((y_true - y_pred)**2) / (2 * y_true.shape[1])
    
    # Optional: Add L1 and L2 regularization terms (commented out by default)
    # l1 = model.get('l1', 0)
    # l2 = model.get('l2', 0)
    # L1_term = L1_reg(l1, W1, W2)
    # L2_term = L2_reg(l2, W1, W2)
    # cost = cost + L1_term + L2_term
    
    return cost

def L2_reg(lambda_val, W1, W2):
    """
    Computes the L2 regularization cost.
    
    Args:
        lambda_val (float): Regularization strength.
        W1 (np.array): Weights matrix for the hidden layer.
        W2 (np.array): Weights matrix for the output layer.
        
    Returns:
        float: The L2 regularization cost.
    """
    # Exclude bias terms (first column) from regularization
    cost = (lambda_val / 2.0) * (np.sum(W1[:, 1:]**2) + np.sum(W2[:, 1:]**2))
    return cost

def L1_reg(lambda_val, W1, W2):
    """
    Computes the L1 regularization cost.
    
    Args:
        lambda_val (float): Regularization strength.
        W1 (np.array): Weights matrix for the hidden layer.
        W2 (np.array): Weights matrix for the output layer.
        
    Returns:
        float: The L1 regularization cost.
    """
    # Exclude bias terms (first column) from regularization
    cost = (lambda_val / 2.0) * (np.sum(np.abs(W1[:, 1:])) + np.sum(np.abs(W2[:, 1:])))
    return cost


# **2. MLP Architecture Functions**

This section defines the core functions for the MLP's architecture: weight initialization, forward propagation to compute predictions, backpropagation to compute gradients, and a prediction utility.

In [ ]:
def MLP_initialize_weights(model):
    """
    Initializes weights for a two-layer MLP with values uniformly distributed in [-1, 1].
    Bias terms are included in the weight matrices as the first column.
    
    Args:
        model (dict): Dictionary containing network dimensions (n_hidden, n_features, n_output).
        
    Returns:
        tuple: W1 (np.array) - Weights for hidden layer, W2 (np.array) - Weights for output layer.
    """
    n_hidden = model['n_hidden']
    n_features = model['n_features']
    n_output = model['n_output']
    
    # W1: (n_hidden x (n_features + 1)) matrix, including bias for hidden layer
    # The +1 accounts for the bias term which will be multiplied by a constant 1 in the input A0
    W1 = 2 * np.random.rand(n_hidden, n_features + 1) - 1
    
    # W2: (n_output x (n_hidden + 1)) matrix, including bias for output layer
    # The +1 accounts for the bias term which will be multiplied by a constant 1 in the activated hidden layer A1
    W2 = 2 * np.random.rand(n_output, n_hidden + 1) - 1
    
    return W1, W2

def MLP_MSELIN_forward(P_input, W1, W2):
    """
    Performs the forward pass through the MLP.
    
    Args:
        P_input (np.array): Input patterns, where each row is an observation (BatchSize x n_features).
        W1 (np.array): Weights matrix for the hidden layer (n_hidden x (n_features + 1)).
        W2 (np.array): Weights matrix for the output layer (n_output x (n_hidden + 1)).
        
    Returns:
        tuple: rA2 (np.array) - Output layer activations (predictions),
               A1 (np.array) - Hidden layer activations with bias,
               A0 (np.array) - Input patterns with bias,
               rZ1 (np.array) - Hidden layer pre-activations.
    """
    batch_size = P_input.shape[0]
    
    # A0: Input patterns with bias term (column of ones) appended
    # Shape: (n_features + 1) x BatchSize
    A0 = np.vstack((np.ones(batch_size), P_input.T))
    
    # rZ1: Pre-activation values for the hidden layer
    # W1: n_hidden x (n_features + 1)
    # A0: (n_features + 1) x BatchSize
    # rZ1: n_hidden x BatchSize
    rZ1 = W1 @ A0
    
    # rA1: Activated values of the hidden layer
    # Shape: n_hidden x BatchSize
    rA1 = MLP_sigmoid(rZ1)
    
    # A1: Activated hidden layer values with bias term appended
    # Shape: (n_hidden + 1) x BatchSize
    A1 = np.vstack((np.ones(batch_size), rA1))
    
    # rA2: Output layer activations (predictions)
    # W2: n_output x (n_hidden + 1)
    # A1: (n_hidden + 1) x BatchSize
    # rA2: n_output x BatchSize
    rA2 = W2 @ A1
    
    return rA2, A1, A0, rZ1

def MLP_MSELIN_backprop(rA2, A1, A0, rZ1, Y_true, W1, W2):
    """
    Computes the partial derivatives of the loss with respect to the weight matrices W2 and W1
    using the backpropagation algorithm.
    
    Args:
        rA2 (np.array): Output layer activations (predictions) (n_output x BatchSize).
        A1 (np.array): Hidden layer activations with bias ( (n_hidden + 1) x BatchSize).
        A0 (np.array): Input patterns with bias ( (n_features + 1) x BatchSize).
        rZ1 (np.array): Hidden layer pre-activations (n_hidden x BatchSize).
        Y_true (np.array): True target values (n_output x BatchSize).
        W1 (np.array): Weights matrix for the hidden layer (n_hidden x (n_features + 1)).
        W2 (np.array): Weights matrix for the output layer (n_output x (n_hidden + 1)).
        
    Returns:
        tuple: delta_W1_unscaled (np.array) - Gradient for W1,
               delta_W2_unscaled (np.array) - Gradient for W2.
    """
    batch_size = A0.shape[1]

    # Step 1: Compute dL_dZ2 (error signal at the output layer pre-activation)
    # This is the derivative of the MSE loss with respect to the output layer's pre-activation Z2.
    # For MSE and linear output, dL/dZ2 = (rA2 - Y_true)
    # Shape: n_output x BatchSize
    dL_dZ2 = rA2 - Y_true
    
    # Step 2: Compute dL_dW2 (gradient for W2)
    # dL/dW2 = dL/dZ2 * A1.T (matrix multiplication)
    # W2: n_output x (n_hidden + 1)
    # dL_dZ2: n_output x BatchSize
    # A1.T: BatchSize x (n_hidden + 1)
    # dL_dW2: n_output x (n_hidden + 1)
    dL_dW2 = dL_dZ2 @ A1.T
    
    # Step 3: Compute dL_dA1 (error signal propagated back to hidden layer activations)
    # dL/dA1 = W2.T * dL/dZ2 (matrix multiplication)
    # W2.T: (n_hidden + 1) x n_output
    # dL_dZ2: n_output x BatchSize
    # dL_dA1: (n_hidden + 1) x BatchSize
    dL_dA1 = W2.T @ dL_dZ2
    
    # Step 4: Compute dL_drZ1 (error signal at hidden layer pre-activation)
    # This involves the derivative of the sigmoid activation function at the hidden layer.
    # dL/drZ1 = (dL/dA1 excluding bias) * sigmoid_derivative(rZ1) (element-wise multiplication)
    # dL_dA1[1:, :] extracts error signal for actual hidden units (excluding bias row)
    # sigma_prime_of_rZ1: n_hidden x BatchSize
    # dL_drZ1: n_hidden x BatchSize
    sigma_prime_of_rZ1 = MLP_sigmoid_derivative(rZ1)
    dL_drZ1 = dL_dA1[1:, :] * sigma_prime_of_rZ1
    
    # Step 5: Compute dL_dW1 (gradient for W1)
    # dL/dW1 = dL/drZ1 * A0.T (matrix multiplication)
    # W1: n_hidden x (n_features + 1)
    # dL_drZ1: n_hidden x BatchSize
    # A0.T: BatchSize x (n_features + 1)
    # dL_dW1: n_hidden x (n_features + 1)
    dL_dW1 = dL_drZ1 @ A0.T
    
    # Step 6: Apply regularization (if enabled - currently commented out)
    # These are the unscaled gradients before applying learning rate and momentum.
    delta_W1_unscaled = dL_dW1
    delta_W2_unscaled = dL_dW2

    # Example of regularization addition (if l1, l2 were active in model):
    # l1 = model.get('l1', 0)
    # l2 = model.get('l2', 0)
    # delta_W1_unscaled[:, 1:] += W1[:, 1:] * (l1 + l2) # For L1/L2 on non-bias weights
    # delta_W2_unscaled[:, 1:] += W2[:, 1:] * (l1 + l2) # For L1/L2 on non-bias weights

    return delta_W1_unscaled, delta_W2_unscaled

def MLP_MSELIN_predict(P_input, W1, W2):
    """
    Predicts the outputs for given input patterns using the trained MLP.
    
    Args:
        P_input (np.array): Input patterns, where each row is a distinct observation.
        W1 (np.array): Trained weights matrix for the hidden layer.
        W2 (np.array): Trained weights matrix for the output layer.
        
    Returns:
        np.array: Predicted outputs (n_output x BatchSize).
    """
    rA2, _, _, _ = MLP_MSELIN_forward(P_input, W1, W2)
    return rA2


# **3. MLP Training Function**

The `MLP_MSELIN_train` function orchestrates the entire training process. It initializes weights, then iterates through a specified number of epochs, processing data in mini-batches. For each mini-batch, it performs a forward pass, calculates the loss, computes gradients via backpropagation, and updates the weights using Gradient Descent.

In [ ]:
def MLP_MSELIN_train(P_train, y_train, model):
    """
    Trains the Multi-Layer Perceptron model using mini-batch Gradient Descent.
    
    Args:
        P_train (np.array): Training input patterns (N_observations x n_features).
        y_train (np.array): True target values for training (N_observations x n_output).
        model (dict): Dictionary containing model configuration (eta, epochs, minibatches, etc.).
        
    Returns:
        tuple: model (dict) - Updated model dictionary with trained weights and cost history,
               W1 (np.array) - Trained weights for the hidden layer,
               W2 (np.array) - Trained weights for the output layer.
    """
    # Initialize weights
    W1, W2 = MLP_initialize_weights(model)
    
    eta = model['eta']
    epochs = model['epochs']
    minibatches = model['minibatches']
    
    # Initialize cost history
    model['cost_history'] = []
    
    # Momentum parameters (commented out by default)
    # delta_W1_prev = np.zeros_like(W1)
    # delta_W2_prev = np.zeros_like(W2)
    # alpha = model.get('alpha', 0) # Momentum coefficient

    num_observations = P_train.shape[0]
    
    for e in range(1, epochs + 1):
        # Optional: Adaptive learning rate (commented out)
        # decrease_const = model.get('decrease_const', 0)
        # current_eta = eta / (1 + decrease_const * e)
        current_eta = eta # Use fixed learning rate if adaptive is off

        # Shuffle data for each epoch
        shuffled_indices = np.random.permutation(num_observations)
        P_shuffled = P_train[shuffled_indices, :]
        y_shuffled = y_train[shuffled_indices, :]

        # Reshape indices for mini-batching
        # np.array_split handles cases where num_observations is not perfectly divisible by minibatches
        mini_batch_indices = np.array_split(np.arange(num_observations), minibatches)
                          
        for m_idx, idx in enumerate(mini_batch_indices):
            # Extract mini-batch data
            P_mini_batch = P_shuffled[idx, :]
            y_mini_batch = y_shuffled[idx, :].T # Transpose y_mini_batch to match expected 1xBatchSize or n_output x BatchSize
            
            # Forward pass
            rA2, A1, A0, rZ1 = MLP_MSELIN_forward(P_mini_batch, W1, W2)
            
            # Compute cost
            cost = MLP_MSE_cost(y_mini_batch, rA2, model, W1, W2)
            model['cost_history'].append(cost)
            
            # Print progress
            print(f'Epoch {e}/{epochs}, Minibatch {m_idx+1}/{minibatches}, Loss (MSE) {cost:.6f}')
            
            # Compute gradient via backpropagation
            delta_W1_unscaled, delta_W2_unscaled = MLP_MSELIN_backprop(rA2, A1, A0, rZ1, y_mini_batch, W1, W2)
            
            # Update parameters using Gradient Descent
            delta_W1 = current_eta * delta_W1_unscaled
            delta_W2 = current_eta * delta_W2_unscaled
            
            W1 = W1 - delta_W1
            W2 = W2 - delta_W2
            
            # Optional: Apply momentum (commented out)
            # W1 = W1 - (delta_W1 + (alpha * delta_W1_prev))
            # W2 = W2 - (delta_W2 + (alpha * delta_W2_prev))
            # delta_W1_prev = delta_W1
            # delta_W2_prev = delta_W2
            
    model['W1'] = W1
    model['W2'] = W2
    return model, W1, W2


# **4. Model Definition and Training Example**

This section sets up the MLP model parameters, generates a synthetic dataset for a simple regression problem, trains the MLP using the `MLP_MSELIN_train` function, and visualizes the training loss and the model's predictions against the true data.

In [ ]:
# Define model parameters
model = {
    'n_features': 1,   # Number of input features
    'n_hidden': 10,    # Number of neurons in the hidden layer
    'n_output': 1,     # Number of output neurons
    'eta': 0.01,       # Learning rate
    'epochs': 500,     # Number of training epochs
    'minibatches': 10, # Number of mini-batches per epoch
    # 'alpha': 0.9,    # Momentum coefficient (uncomment if using momentum)
    # 'l1': 0.001,     # L1 regularization strength (uncomment for L1)
    # 'l2': 0.001      # L2 regularization strength (uncomment for L2)
}

# Generate synthetic data for a regression problem
num_samples = 200
X_train = np.linspace(-5, 5, num_samples).reshape(-1, 1) # Input feature
y_train = np.sin(X_train) + 0.2 * np.random.randn(num_samples, 1) # True output with noise

# Train the MLP model
print("\nStarting MLP training...")
trained_model, W1_trained, W2_trained = MLP_MSELIN_train(X_train, y_train, model)
print("MLP training complete.")

# Plot the training loss history
plt.figure(figsize=(10, 6))
plt.plot(trained_model['cost_history'], label='Training Loss (MSE)', color='blue')
plt.xlabel('Mini-batch Iteration')
plt.ylabel('Loss (MSE)')
plt.title('MLP Training Loss Over Iterations')
plt.grid(True)
plt.legend()
plt.show()

# Make predictions on the training data
y_pred_train = MLP_MSELIN_predict(X_train, W1_trained, W2_trained).T # Transpose to match y_train shape

# Plot original data and MLP predictions
plt.figure(figsize=(10, 6))
plt.scatter(X_train, y_train, label='True Data', color='gray', alpha=0.7, s=20)
plt.plot(X_train, y_pred_train, label='MLP Predictions', color='red', linewidth=2)
plt.xlabel('Input Feature')
plt.ylabel('Output Value')
plt.title('MLP Regression: True vs. Predicted Values')
plt.grid(True)
plt.legend()
plt.show()
